<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_2/04_image_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ホップフィールドネットワークによる手書き数字（MNIST）の復元
ホップフィールドネットワークを使用して、手書き数字（MNIST）データセット の画像を復元する例を紹介します。  
MNISTデータセットは、28x28ピクセルの手書き数字（0から9）の画像から成りますが、これを復元することで、ホップフィールドネットワークの能力を実証します。

## コード：ホップフィールドネットワークを使った手書き数字の復元
このコードでは、以下のことを行います：
1. **MNISTデータセット** から手書き数字の画像を読み込みます。
2. 数字パターン（例: 数字「0」）をホップフィールドネットワークに学習させます。
3. ランダムなノイズではなく、**画像の一部（四角形の領域）を欠落** させます。
4. 欠損部分がある画像を入力し、ホップフィールドネットワークで復元を試みます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import binarize

class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons))

    def train(self, patterns):
        """パターンを学習し、重み行列を構築する"""
        for p in patterns:
            p = p.reshape(self.num_neurons, 1)
            self.weights += np.dot(p, p.T)
        np.fill_diagonal(self.weights, 0)

    def recall(self, pattern, steps=10):
        """パターンを再現し、欠損を補完する"""
        result = pattern.copy()
        for _ in range(steps):
            result = np.sign(np.dot(self.weights, result))
        return result

def add_missing_part(pattern, mask_shape=(10, 10)):
    """画像の一部を欠落させる"""
    corrupted_pattern = pattern.copy()
    height, width = 28, 28
    mask_height, mask_width = mask_shape
    # 欠損領域の開始位置をランダムに選択
    top = np.random.randint(0, height - mask_height)
    left = np.random.randint(0, width - mask_width)
    # 欠損部分を -1 で埋める（黒にする）
    corrupted_pattern[top*width + left : top*width + left + mask_width] = -1
    for row in range(1, mask_height):
        start_index = (top + row) * width + left
        corrupted_pattern[start_index : start_index + mask_width] = -1
    return corrupted_pattern

def plot_image(vector, title, image_shape=(28, 28)):
    """1次元配列を画像としてプロット"""
    plt.imshow(vector.reshape(image_shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# 1. MNISTデータセットの読み込み
(x_train, y_train), (_, _) = mnist.load_data()

# 2. データの前処理（2値化）
x_train_bin = binarize(x_train.reshape(-1, 784), threshold=127).astype(int) * 2 - 1

# 3. 学習用のパターンを準備（数字「0」と「1」を使用）
pattern_zero = x_train_bin[y_train == 0][0]  # 数字「0」のパターン
pattern_one = x_train_bin[y_train == 1][0]   # 数字「1」のパターン

# 4. ホップフィールドネットワークの初期化と学習
network = HopfieldNetwork(num_neurons=784)
network.train([pattern_zero, pattern_one])

# 学習に使用したパターンを表示
plot_image(pattern_zero, "Training Pattern: 0")
plot_image(pattern_one, "Training Pattern: 1")

# 5. 画像の一部を欠落させる（数字「0」）
missing_pattern = add_missing_part(pattern_zero, mask_shape=(10, 10))
plot_image(missing_pattern, "Pattern with Missing Part: 0")

# 6. 欠損部分の復元（パターンの再現）
recalled_pattern = network.recall(missing_pattern)
plot_image(recalled_pattern, "Recalled Pattern (Filled): 0")

### コードの解説

#### 1. **`HopfieldNetwork` クラス**
- **`train` メソッド**:
  - 与えられたパターンの **外積** を使って重み行列を構築し、学習します。
- **`recall` メソッド**:
  - 欠損部分があるパターンを入力して、学習済みパターンに収束させ、欠損部分を復元します。

#### 2. **`add_missing_part` 関数**
- **画像の一部を欠落** させます。
  - 欠損領域のサイズ（`mask_shape`）を指定して、ランダムな位置に四角形の欠損領域を作成します。
  - 欠損部分はピクセル値 `-1`（黒）で埋めます。

#### 3. **`plot_image` 関数**
- 1次元配列を28×28ピクセルの画像としてプロットします。

#### 4. **データの前処理**
- `MNIST` データセットを読み込み、**2値化** します。
  - ピクセル値を `0` または `1` に変換し、その後 `-1` と `1` の値に調整します。

#### 5. **学習と復元の流れ**
- **学習パターン** をホップフィールドネットワークに学習させます。
- **欠損部分** を持つパターンを生成し、その画像を表示します。
- ホップフィールドネットワークで **欠損部分を復元** し、再現された画像を表示します。

---

### 実行結果の例

1. **Training Pattern: 0**
   - 学習に使用した数字「0」の画像。
  
2. **Training Pattern: 1**
   - 学習に使用した数字「1」の画像。

3. **Pattern with Missing Part: 0**
   - 数字「0」の画像の一部（10×10ピクセル）が欠落した状態。

4. **Recalled Pattern (Filled): 0**
   - 欠損部分がホップフィールドネットワークによって復元された画像。

---

### まとめ
- ホップフィールドネットワークは、画像の一部が欠落している場合でも、**学習したパターン** を基にして元の画像に近づけることができます。
- 今回の例では、`MNIST` の手書き数字を使って、欠損部分を復元しました。
- この手法は、ノイズ除去や部分欠損からの補完など、パターンの補完能力が必要なタスクに応用できます。

このコードをさらに拡張すれば、より複雑なパターンや画像の補完にも応用可能です。